#### Note: The following has been tested successfully on NU Discovery cluster

### 1. Connect to the Discovery cluster



### 2. Install/load Anaconda

We have two options here (I prefer the 2nd):
<a id='1st'></a>
### 2 (1st option) Obtain and install Miniconda executable specifically for python 2.7
#### 2.1 Obtain
```bash
$ wget https://repo.anaconda.com/miniconda/Miniconda2-py27_4.8.3-Linux-x86_64.sh
```
#### 2.2 Install
```bash
$ bash Miniconda2-py27_4.8.3-Linux-x86_64.sh
```
### or

<a id='2nd'></a>
### 2 (2nd option). Load Anaconda module and activate it
```bash
$ module load anaconda2/2018.12
$ echo ". /shared/centos7/anaconda2/2018.12/etc/profile.d/conda.sh" >> ~/.bashrc
$ source ~/.bashrc
```

### 3. Reserve a gpu/multigpu compute node

```bash
$ srun -p gpu --nodes=1 --pty --gres=gpu:1 --time=04:00:00 --export=ALL /bin/bash
```

### 4. Activate the environment
If you went with the [first option](#1st):

```bash
$ source miniconda2/bin/activate
```

If you went with the [second option](#2nd):
```bash
$ conda activate
```
If everything goes well, `(base)` will show up next to your `username@computenode` in shell, as

```bash
(base)[username@c0001]$
```

### 5. Load the required modules

```bash
$ module load boost/1.63.0
$ module load cuda/10.2
```

### 6. Create a conda environemnt for PyGBe (takes a while)

First, delete the previously created `pygbe-env` environment from the old installation. You might have named it differently. If that's the case, you can see your environments at

```bash
$ ls ~/.conda/envs
```
#### To remove the old environment
```bash
$ conda remove -n pygbe-env --all
```
or delete the env folder
```bash
$ rm -rf ~/.conda/envs/pygbe-env
```
#### To create the new environment
```bash
$ conda create -n pygbe-env python=2.7
```

### 7. Activate the PyGBe environment
```bash
$ conda activate pygbe-env
```

### 8. Install the remaining dependencies

#### 8.1 `numpy` and `scipy`

```bash
$ conda install numpy scipy
```

#### 8.2 `pycuda`
before installing pycuda, we want to make sure there is absolutely no trace of previous installations left on your machine. So we delete them from the potential locations.
```bash
$ rm -rf ~/.conda/pkgs/pycuda*
$ rm -rf ~/.local/lib/python2.7/site-packages/pycuda*
```
Now install `pycuda`
```bash
$ pip install -Iv pycuda
```

### 9. Clone the repository

```bash
$ git clone https://github.com/alineu/pygbe.git
```
or
```bash
$ git clone git@github.com:alineu/pygbe.git
$ cd pygbe
```

### 10. checkout the asymmetric branch

```bash
$ git checkout asymmetric
```

### 11. Download and install SWIG
```bash
$ mkdir src
$ cd src
$ wget https://versaweb.dl.sourceforge.net/project/swig/swig/swig-3.0.12/swig-3.0.12.tar.gz
$ tar -xvzf swig-3.0.12.tar.gz
$ cd swig-3.0.12
$ ./configure --prefix=$PWD
$ make
$ make install
```

### 12. Install PyGBe and test the installation

#### 12.1 Install
```bash
$ cd ../../bem_pycuda/
$ make all
```
#### 12.2 Test
```bash
$ python main_asymmetric.py input_files/his.param input_files/his_stern.config --asymmetric --chargeForm
```
If the library is installed properly, you should see something like

```bash
Run started on:
	Date: year/m/d
	Time: hr:min:sec

Reading pqr for region 2 from ../geometry/his/his_prot.pqr

Reading surface 0 from file ../geometry/his/his_d01
Time load mesh: 0.036644
Removed areas=0: 0

Reading surface 1 from file ../geometry/his/his_d01_stern
Time load mesh: 0.041806
Removed areas=0: 81

Total elements : 1555
Total equations: 3110

...
...
...

Totals:
Esolv = -19.045255 kcal/mol
Esurf = 0.000000 kcal/mol
Ecoul = -119.087367 kcal/mol

Time = 15.916039 s
```

### 13. Whenever you're done:

```bash
$ conda deactivate
$ conda deactivate
```

twice!

### Note
Next time, when you connect to a GPU compute node you only need to run the following to enable and use `PyGBe` library:

#### Activate the Anaconda Module
If you installed Miniconda locally (via [first option](#1st)):
```bash
$ source activate miniconda2
```
otherwise, 
```bash
$ module load anaconda2/2018.12
$ conda activate
```

#### Activate the PyGBe Environment

```bash
$ conda activate pygbe-env
```

#### Load the Rest of the Modules
```bash
$ module load boost/1.63.0
$ module load cuda/10.2
```
The library should be ready to use!

#### Important
You might have to add the location of `pycuda` library to your `PATH` if `Python` fails to import the library. You can do this by adding
```bash
export PATH=$PATH:$HOME/.conda/envs/pygbeenv/lib/python2.7/site-packages/pycuda
```
to your `~/.bashrc` file (and then source it!)

If you don't know the location of `pycuda` library you can find it using
```bash
find ~ -type d -name 'pycuda*'
```